In [81]:
from selenium.webdriver import Chrome
import time
import os

url = "https://www.cathaybk.com.tw/cathaybk/personal/credit-card/cards/intro/list/"

# 開啟並取得國泰信用卡網頁。
driver = Chrome("./chromedriver.exe")
# 注意是否要放大，或自訂螢幕大小。
driver.maximize_window()
time.sleep(1)
driver.get(url)

In [82]:
#滾輪往下滾
def scrolltop(pos):
    js = "document.documentElement.scrollTop=%s" % pos
    driver.execute_script(js)
    
#滾輪滾到底
def scrolldown():
    js = "document.documentElement.scrollTop=10000"
    driver.execute_script(js)
    
#去到下一頁
def backpage(page):
    driver.find_element_by_id(
        "layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_{}".format(page-1)).click()
    # 給他三秒鐘的時間跑
    time.sleep(3)
    
#抓取網頁內的資料
def catch():
    #先判斷網頁有幾個上標
    title = []
    for num in range(10):
        try:
            lenth = driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(num+1)).text
            #確保串列內字元不為空
            if lenth != "":
                title.append(lenth)
            else:
                continue
        except:
            break
    # 根據上標數做迴圈
    for item in range(len(title)):
        try:
            driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 1)).click()
            data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
            print(data.text)
            print("--------------------------------------")
        except:
            #這裡要新增卡友權益
            continue
    # 上一頁到卡片清單頁
    for item2 in range(len(title)):
        driver.back()

In [83]:
# 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
page = 1
while page < 5:
    pos = 200
    for i in range(10):
        pos += 220
        scrolltop(pos)
        # 注意抓取的值為list，要一個一個取出。
        driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
        catch()
        print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
        time.sleep(1)
        if page == 1:
            driver.back()
            scrolltop(pos)
            time.sleep(1)
        else:
            driver.back()
            scrolldown()
            backpage(page)
    page += 1
    backpage(page)
print("好像沒東西喔!!!")

新卡滿額禮
世界卡2019/7月-2019/10月新卡刷卡禮
2019/7/1-2019/10/31期間首次申辦世界卡正卡，核卡日後60天內刷一般消費累積達門檻，享精選好禮回饋:
<滿額禮>     
精緻滿額禮：達NT$15萬以上，「Samsonite Polygon 25吋行李箱 (藍)」乙個。
尊榮滿額禮：達NT$60萬以上，「紅利積點30萬點」。
<加碼禮>     
加碼禮：達NT$18萬以上，「兩次免費機場接或送禮遇」。

贈品介紹
  精緻滿額禮：Samsonite Polygon 25吋行李箱(藍)        
材質：100%PC (聚碳酸酯)
尺寸：47 X 69 X 31 cm
淨重：4 Kg 容量：77 L
保修期間：3年 (相關保修規範以原廠官網說明為準)
商品詢問及維修電話 : 0800-088-349
商品簡介：
採用 FLAT-MAX™ 結構設計「面蓋與主箱體為 20:80 比例」，行李箱20%是拉桿系統的面蓋，80%是主要儲存空間的主箱體，打造出特大容量。
箱蓋側面有煞車止滑開關，選用日本 Hinomoto 的EASY BRAKE™煞車輪，拉桿兩側的輪子在煞車系統啟動時，仍可拖行。
TSA 海關鎖+隱藏式地址貼牌。                    
尊榮滿額禮：紅利積點30萬點
紅利積點30萬點可兌換高額航空里程/飯店積分、百貨禮券、商品，讓您輕鬆旅行、購物，盡享寫意自在品味生活。(瞭解更多「舞動紅利」紅利點數兌換辦法)
  紅利積點30萬點可兌換多樣化優惠：
亞洲萬里通74,000里數，或長榮航空60,000哩，或Le Club AccorHotels雅高樂雅會獎勵積分20,000點
新光三越百貨商品禮券17,000元，或遠東SOGO百貨商品券17,500元
TreeMall紅利線上購物全站超過10萬種品項
加碼禮：兩次免費機場接或送禮遇
享指定區域至以下機場免費機場接或送禮遇2次。
台北松山機場
桃園國際機場
台中國際機場
高雄國際機場(小港機場)
24小時服務預約專線 (04) 2206-1608
(持卡人倘符合消費資格，本行將透過簡訊通知，持卡人收到簡訊後才可進行預約。)
(本免費機場接送禮遇有限定服務區域，詳細服務適用條件與限制依官網公告為準。)


注意事項









生日貴賓尊寵禮遇
Sisley
生日

申辦資格
年滿20歲以上可申請正卡。
附卡申請人需為正卡申請人之父母、兄弟姊妹、配偶、配偶之父母或年滿15歲之子女。
附卡申請人如未滿20歲，須法定代理人(即父母/監護人)共同簽名同意。現職需滿一年且年收入達NT$200萬。

年費說明
正卡：NT$20,000
項目 免年費條件
首年度 正卡人首次申辦世界卡且於本行近三個月平均往來資產達 NT$300 萬元(含)以上且於當月份專案名單內。
每戶限回饋一次。
次年度起 正、附卡累計年消費達 NT$80 萬(含)以上者，可享次年免年費。
正、附卡累計年消費達 NT$40 萬(含)以上者，可享次年年費 5 折優惠。
備註 附卡免年費

卡面介紹
全新卡面 原卡面


申辦方式
1.已為本行信用卡/活存(儲)帳戶之客戶，立即線上申辦，免簽名寄回。
2.未與本行有業務往來者:
線上申辦：
1. 具有他行信用卡，立即線上申辦，免簽名寄回。
2. 非上述者，線上完成資料填寫後，請列印簽名檢附申請文件寄回本行申請。
臨櫃申辦：
需備妥身分證正、反面影本及扣繳憑單或近三個月薪資入帳存摺之內頁及封面影本，至鄰近分行辦理。 查詢分行據點
--------------------------------------
現在到第幾頁: 1 到第幾張卡片: 1
現在到第幾頁: 1 到第幾張卡片: 2
新戶首刷禮享OPENPOINT
2019/7/1~2019/12/31，新戶申辦KOKO icash聯名卡正卡，並於核卡後30天內刷一般消費(單筆須滿NT$888)或icash自動加值，合計滿3筆，享首刷禮150,000點OPENPOINT！
* KOKO COMBO icash聯名卡不適用本活動。 
 

注意事項

指定網購/超商/交通通路 刷卡享3%現金回饋
108/12/31前，完成登錄後，當期帳單KOKO信用卡新增一般消費NT$5,000(含)以上，當期KOKO icash聯名卡＆KOKO(COMBO)悠遊聯名卡之指定網購/超商/交通通路消費金額享3%現金回饋！(每戶每月帳單回饋上限NT$300)
指定網購通路：APPLE iTunes、APPLE之App Store、Google Play、淘寶網、TreeMall(泰贈點紅利網)、Yahoo!奇摩購物中心/超級商城/拍賣、PChome線上購物/商店街、露天拍賣、博客來網路書店、udn買

機場貴賓室禮遇
免費6次/年貴賓室禮遇
108/1/1~108/12/31
世界卡最高享免費龍騰機場貴賓室禮遇6次/年
申請方式：可透過本行專屬網頁或電話申請。
申請資料：請準備您的護照英文姓名、行動電話號碼、有效聯名卡正卡卡號及生日。
使用方式：以手機出示您的龍騰卡卡號，使用前請先下載龍騰出行APP，使用時出示您手機上的龍騰卡卡號或掃描您手機上的二維碼使用 (使用前請確認龍騰卡已綁定有效信用卡卡號)。

貴賓室優惠價NT$700起
108/1/1~108/12/31
鈦商卡及白金卡卡友可享優惠價NT$700起
申請方式：可透過本行專屬網頁或電話申請。
申請資料：請準備您的護照英文姓名、行動電話號碼、有效聯名卡正卡卡號及生日。
使用方式：以手機出示您的龍騰卡卡號，使用前請先下載龍騰出行APP，使用時出示您手機上的龍騰卡卡號或掃描您手機上的二維碼使用 (使用前請確認龍騰卡已綁定有效信用卡卡號)。
※以上服務由肯驛國際股份有限公司提供
04-2206-8195
https://www.dragonpass.com.tw/cathaybank/


注意事項

免費機場接送禮遇
一年免費2次機場接送禮遇
108/1/1~108/12/31
世界卡卡友享指定區域免費機場接送禮遇2次/年
國際機場 地區
松山機場 台北市、新北市、基隆市、桃園市、新竹縣市
桃園國際機場 台北市、新北市、基隆市、桃園市、新竹縣市、苗栗縣市、台中市、彰化縣市 、 南投縣市、宜蘭縣 、花蓮縣市
台中清泉岡國際機場 新竹縣市、苗栗縣市、台中市、彰化縣市、南投縣市、雲林縣、嘉義縣市
高雄國際航空站
（原小港機場） 台中市、彰化縣市、南投縣市、雲林縣、嘉義縣市、台南市、高雄市、屏東縣市、台東縣市
優惠 消費門檻
免費禮遇
108/4/1~108/12/31期間預約服務者，預約前須持聯名卡指定卡別刷達下列任一門檻：
3個月內機票或旅遊團費新增消費合計達NT$10,000
前期帳單或當期帳單新增消費達NT$20,000
前三期帳單合計新增消費達NT$60,000
(本免費機場接送禮遇有限定服務區域，詳細服務適用條件與限制依官網公告為準。)
  立即預約

機場接送卡友獨享優惠
108/1/1~108/12/31
鈦商卡/白金卡/里享卡卡友享機場接送優惠價NT$428元起
立即預約

海外網路樂遊
免費Wi-F

精選住房優惠NT$3,600起
極致無限卡、無限卡專屬
HOTEL COZZI和逸系列飯店
．台北民生館-「舒適客房」
．台北忠孝館-「舒適客房」
．台南西門館-平日(週日至週五)「舒適客房」，假日(週六及特定日期)「舒適客房」享優惠價NT$5,100、
．高雄中山館-平日(週日至週五)「舒適套房」，假日(週六及特定日期)「景隅客房」
預約專線：(02)7706-3600
(含服務費；不含早餐)

麗緻旅館系統飯店
．台北亞都麗緻大飯店─「卓越客房」
預約專線：(02)2598-3455
．台中亞緻大飯店─「標準客房」
預約專線：(04)3700-3839
．台南大億麗緻酒店─「雅緻雙人房」
預約專線：(06)216-0077
(含服務費及兩客早餐)

國賓大飯店系列
．台北國賓大飯店─精緻客房
預約專線(02)2551-1111轉訂房組
．新竹國賓大飯店─豪華客房
預約專線(03)515-1111 轉訂房組
．高雄國賓大飯店─豪華客房(年度專案價NT$3,300)
預約專線(07)211-5211 轉訂房組
(含服務費及兩客早餐)


注意事項

全球長榮國際連鎖酒店住房款待
台灣住房每晚NT$3,600起
長榮桂冠酒店（基隆）- 平日「海景客房」
訂房專線：(02) 2429-8827
長榮桂冠酒店（台北）- 「高級客房」
訂房專線：(02) 2429-8827
長榮桂冠酒店（台中）- 「標準樓層高級客房」
訂房專線：(04) 2324-2277
台糖長榮酒店（台南）- 平日「豪華雙床房」
訂房專線：(06) 289-6688
長榮鳯凰酒店（礁溪）-預訂季節性住房專案，享房型升等並加贈精選紅酒一瓶
訂房專線：(03) 910-0688
(含服務費，長榮桂冠酒店（基隆、台中、台南）含兩客早餐)

全球住房優惠
長榮桂冠酒店（曼谷）- 「豪華客房」每房每晚泰銖2942.5元
訂房專線：+66-2-266-9988
長榮桂冠酒店（檳城）- 「高級客房」每房每晚馬幣323元起
訂房專線：+60-4-222-9988
長榮桂冠酒店（上海）- 「標準客房」每房每晚人民幣688元起
訂房專線：+86-21-8016-9988
長榮桂冠酒店（巴黎）- 「高級客房」每房每晚歐元175元起
訂房專線：+33-1-4758-8899
(含服務費及早餐)
(全球訂房專線+886-2-2

御璽卡、無限卡消費滿額 享高鐵88折優惠
2019/1/1~2019/12/31 御璽卡、無限卡持卡人前月帳單新增消費達NT$15,000(含)以上，當月至台灣高鐵網路購票系統之「信用卡合作優惠專區」、手機T-EX行動購票APP「信用卡優惠專區」、或臨櫃刷卡購買4日內之標準車廂車票，享票價88折之優惠(每月優惠上限四張)。


高鐵88折優惠注意事項


機場接送禮遇
無限卡機場接送服務
活動期間：2019/1/1～2019/12/31
活動條件：刷卡購買國際線機票全額或國外團費8成以上且付款成功者
活動優惠：享機場接送每趟380元起
04-2206-9342
立即預約
※詳情以卡友權益頁面所載資訊為主

御璽卡機場接送服務
活動期間：2019/1/1～2019/12/31
活動條件：刷卡購買國際線機票全額或國外團費8成以上且付款成功者
活動優惠：享機場接送每趟588元起
04-2206-4655
本行卡友獨享：由本行為御璽卡持卡人支付電話預約每筆訂單處理費NT$50/次

立即預約
※詳情以卡友權益頁面所載資訊為主

白金卡機場接送服務
活動期間：2019/1/1～2019/12/31
活動條件：刷卡購買國際線機票全額或國外團費8成以上且付款成功者
活動優惠：享機場接送每趟799元起
04-2206-2550
立即預約


注意事項


機場外圍免費停車
無限卡免費30天機場外圍停車優惠
活動期間：2019/1/1~2019/12/31
活動內容：刷Costco無限卡支付當次80%以上之國外旅遊團費、或國際線機票全額票款，且單筆刷卡金額滿NT$1萬元（含）以上，享全年不限次數之國際機場外圍免費停車30天/次。
※各停車場車位有限，實際車位以現場為準，停滿為止，詳情及注意事項請見卡友權益

御璽卡、白金卡免費７天機場外圍停車優惠
活動期間：2019/1/1~2019/12/31
活動內容：刷Costco御璽卡、Costco白金卡支付當次80%以上之國外旅遊團費、或國際線機票全額票款，且單筆刷卡金額滿NT$1萬元（含）以上，享全年2次之國際機場外圍免費停車7天/次。
※各停車場車位有限，實際車位以現場為準，停滿為止，詳情及注意事項請見卡友權益

無限卡全球機場貴賓室禮遇
2019/4/1~2019/12/31
使用服務前須以Costco無限卡新增消費達以下任一項門檻，

...

高爾夫球 揮桿禮遇
至Golface官網預約擊球日期並使用鈦金商務卡線上刷卡結帳，即可享有優惠。
球場名稱 優惠價
平日早鳥 平日 假日
台北北海高爾夫鄉村俱樂部 NT$2,380 NT$2,775 NT$3,380
桃園大溪高爾夫俱樂部 NT$3,300 NT$4,000 NT$6,250
 旭陽高爾夫俱樂部 NT$2,350 NT$2,350 NT$3,000
台中高爾夫俱樂部 NT$2,080 NT$2,080 NT$2,380
南投南峰高爾夫俱樂部 NT$2,070 NT$2,550 NT$2,970
 嘉義棕梠湖渡假村高爾夫球場 NT$2,040 NT$2,540 無 
台南高爾夫俱樂部 NT$1,880 NT$2,320 NT$2,590
高雄大崗山高爾夫球場 NT$2,300 NT$2,700 NT$3,300
※以上金額含果嶺、桿弟、球車、保險等金額，台北北海/桃園大溪未滿2客；其餘球場未滿4客須補桿弟、球車差額。
 

注意事項
--------------------------------------
申辦資格
年滿20歲以上可申請正卡。
附卡申請人需為正卡申請人之父母、兄弟姊妹、配偶、配偶之父母或年滿15歲之子女。
附卡申請人如未滿20歲，須法定代理人(即父母/監護人)共同簽名同意。
現職需滿一年且年收入達NT$ 60萬元

年費說明
正卡：NT$1,800
免年費條件
首年度 首年免年費
次年度起 正、附卡年消費達 NT$18萬 以上者。
正、附卡消費合併計算。
備註 附卡免年費

卡面介紹
全新卡面 原卡面

申辦方式
已為本行信用卡/活存(儲)帳戶之客戶，立即線上申辦，免簽名寄回。
未與本行有業務往來者:
線上申辦：
具有他行信用卡，立即線上申辦，免簽名寄回。
非上述者，線上完成資料填寫後，請列印簽名檢附申請文件寄回本行申請。
臨櫃申辦：需備妥身分證正、反面影本及扣繳憑單或近三個月薪資入帳存摺之內頁及封面影本， 至鄰近分行辦理。查詢分行據點
--------------------------------------
現在到第幾頁: 2 到第幾張卡片: 2
尊榮御璽卡刷LINE Pay 拿紅利，回饋率3%！
108/5/13~108/12/31 以LINE Pay 一卡通帳戶連結國泰世華帳戶，並透過LINE Pay使用已綁定的國泰世華尊

臨櫃申辦：需備妥身分證正、反面影本及扣繳憑單或近三個月薪資入帳存摺之內頁及封面影本， 至鄰近分行辦理。 查詢分行據點
--------------------------------------
現在到第幾頁: 2 到第幾張卡片: 4
鈦金商務卡刷LINE Pay 拿紅利，回饋率3%！
108/5/13~108/12/31 以LINE Pay 一卡通帳戶連結國泰世華帳戶，並透過LINE Pay使用已綁定的國泰世華美元雙幣卡消費，登錄享紅利點數10 倍回饋，回饋率最高 3%！（含原卡片回饋，本活動加碼每月歸戶回饋上限 5,000 點）
  舉例說明：活動期間於 LINE Pay 一卡通連結國泰世華帳戶，且透過LINE Pay綁定美元雙幣卡消費NT$1 萬為例，共可獲得紅利點數 5,000 點，回饋率 3%！
項目 回饋內容 紅利點數
原卡片回饋 紅利 1 倍 ( 0.3%回饋率 ) 500 點紅利
加碼回饋 紅利 9 倍 ( 2.7%回饋率 ) 4,500 點紅利
合計 紅利 10 倍 ( 3.0%回饋率 ) 5,000 點紅利
活動登錄

注意事項

外幣換匯優惠
美元雙幣正卡持卡人於108/1/1起，該卡當月帳單一般消費金額達NT$3,000，次月6日後享網銀、行動銀行APP「外匯轉帳及兌換」，指定幣別換匯減碼優惠乙次。
新台幣兌換 優惠匯差
美元 0.035
日幣 0.0015
人民幣 0.015


注意事項

代扣定期定額基金享手續費38折優惠
美元雙幣卡正卡持卡人至網銀選擇信用卡代扣定期定額專案系列基金商品享手續費38折起
新卡友
核卡後60天(含)內，新申購、異動扣款金額或轉換交易，享手續費38折優惠
核卡後61天後優惠同既有卡友

既有卡友
新申購、異動扣款金額或轉換交易，享手續費65折優惠


注意事項
--------------------------------------
海外消費紅利點數最高5倍
108/12/31前，刷卡海外消費紅利點數加倍回饋。
國內刷美元雙幣卡消費20元累積紅利點數1點
國外消費每刷卡US$2就可累積紅利點數3點
國外消費單筆刷卡金額達US$20(含)以上
鈦金商務卡紅利點數加碼5倍
白金卡紅利點數加碼3倍
*加碼倍數含原卡片1倍


注意事項

Mastercard樂賞利
海外指定商店消費享額外現金回饋
即日起~10

國際機場接送禮遇
晶緻卡機場接送服務
活動期間: 108/1/1~108/12/31，服務預約日最晚須於108/12/23當日預約完畢。
活動辦法:刷JCB SOGO晶緻卡，購買國際航線機票或支付國外旅遊團費金額，且於上開刷卡消費日(即為授權碼取得日)後之二個月內，透過預約中心或網路進行預約，可預約服務日期僅為進線(件)日後近三個月內，享機場接送優惠。
單筆達新台幣30,000元(含)以上至新台幣59,999元且付款成功者(若單獨購買國際航線機票須達新台幣30,000元(含)以上)，於本活動期間內可享「指定國際機場接送去程或回程1次免費」
單筆達新台幣60,000元(含)以上且付款成功者(若單獨購買國際航線機票須達新台幣60,000元(含)以上)，於本活動期間 內可享「指定國際機場接送去回程各1次之免費優惠服務」
JCB白金卡或JCB晶緻卡享JCB機場接送服務6折優惠價(價目表詳見JCB活動官網說明)
02-2596-6606
立即預約
※每月合計限量2,500次。詳情以卡友權益頁面及JCB國際組織所載資訊為主

使用須知

鈦金卡機場接送服務
活動期間：108/1/1～108/12/31
活動條件：刷卡購買國外機票或國外團費達金額超過NT$10,000且付款成功者
活動優惠：機場接送每趟628元起
02-7733-0589
立即預約
詳情以卡友權益頁面所載資訊為主

機場外圍免費停車
鈦金卡、晶緻卡、白金卡免費７天機場外圍停車優惠
優惠期間：108/1/1至108/12/31
優惠條件：須刷本行卡支付當次八成以上國外旅遊團費，或國際線機票全額票款，且單筆金額滿新臺幣1萬元(含)，可享免費停車優惠
優惠內容：一年限2次，每次最高免費7天
※各停車場車位有限，實際車位以現場為準，停滿為止
※詳情以卡友權益頁面所載資訊為主

超值旅遊保險保障
持國泰世華銀行信用卡，為自己或家人（限配偶及未滿25足歲之未婚子女）支付公共運輸工具全額票款、或80%以上之旅遊團費，就可享有以下超值保險保障
保險期間：108/1/1 12時 至 109/1/1 12時 止
鈦金卡、白金卡、晶緻卡：搭乘公共運輸工具期間享2000萬死傷、100萬醫療
普卡：搭乘公共運輸工具期間享750萬死傷、10萬醫療
詳情以卡友權益頁面所載資訊為主

道路救援服務
活動期間：於卡片有效期間內完成道路救援車號登錄

現在到第幾頁: 2 到第幾張卡片: 10


ElementClickInterceptedException: Message: element click intercepted: Element <a id="layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_2" data-page="..." href="javascript:__doPostBack('layout_0$rightcontent_0$updatepanel_1$RptPagination$ctl02$HlkPageNumber','')">3</a> is not clickable at point (1070, 916). Other element would receive the click: <div class="f-contactWrap footerWrap">...</div>
  (Session info: chrome=77.0.3865.120)


In [80]:
#先判斷網頁有幾個上標
title = []
for num in range(10):
    try:
        lenth = driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(num+1)).text
        #確保串列內字元不為空
        if lenth != "":
            title.append(lenth)
        else:
            continue
    except:
        break
# 根據上標數做迴圈
for item in range(len(title)):
    try:
        driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 1)).click()
        data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
        print(data.text)
        print("--------------------------------------")
    except:
        #這裡要新增卡友權益
        continue
# 上一頁到卡片清單頁
for item2 in range(len(title)):
    driver.back()
    
###卡片即便只有4個上標,原始碼還是做了7個上標 所以要想辦法讓他只back上標次
###新想法:先判斷text有沒有文字 有就存入list沒有就不存,最後看list長度

新戶首刷禮享OPENPOINT
2019/7/1~2019/12/31，新戶申辦KOKO icash聯名卡正卡，並於核卡後30天內刷一般消費(單筆須滿NT$888)或icash自動加值，合計滿3筆，享首刷禮150,000點OPENPOINT！
* KOKO COMBO icash聯名卡不適用本活動。 
 

注意事項

指定網購/超商/交通通路 刷卡享3%現金回饋
108/12/31前，完成登錄後，當期帳單KOKO信用卡新增一般消費NT$5,000(含)以上，當期KOKO icash聯名卡＆KOKO(COMBO)悠遊聯名卡之指定網購/超商/交通通路消費金額享3%現金回饋！(每戶每月帳單回饋上限NT$300)
指定網購通路：APPLE iTunes、APPLE之App Store、Google Play、淘寶網、TreeMall(泰贈點紅利網)、Yahoo!奇摩購物中心/超級商城/拍賣、PChome線上購物/商店街、露天拍賣、博客來網路書店、udn買東西、momo購物網、PayEasy、17Life、GOMAJI、91app、生活市集(含原3C市集)、瘋狂賣客、Pinkoi、ASOS、好吃市集、FunNow、ibon mart。
指定超商：全台四大超商實體門市(7-11、全家、萊爾富、OK) ，不含代收費用。
指定交通：台鐵、台灣高鐵、Uber、台灣大車隊、WeMo Scooter。
立即登錄



注意事項


搭乘捷運/公車滿20次 享15,000點OPENPOINT回饋
活動期間：108/7/1~108/12/31
活動規則：凡活動期間內持KOKO(COMBO)icash聯名卡，單卡每月使用icash功能扣款搭乘捷運/公車滿20次，享15,000點OPENPOINT回饋。(須登錄，不累贈，每戶每月限回饋一次)
立即登錄



注意事項

國賓/台中新光影城6折起
108/12/31前，刷KOKO icash聯名卡購買當日電影票享優惠
憑一個月內同卡號之單筆NT$1,000(含)以上實體刷卡簽單，享週一～週四6折，週五～週日及國定假日63折起優惠！
平假日不限時段，全台國賓/台中新光影城刷卡購票享優惠價NT$250起！
於國賓影城網路訂票，還享免付取票手續費NT$20優惠！
立即訂票去 » 國賓網路訂票系統


注意事項

KKBOX會員67折優惠
108/12/